In [31]:
#データの読み込みから実行まで全てを実行するくRunnerクラスを定義する
from typing import Any, Dict, Tuple, Union
import sys
import numpy as np
import joblib
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import Callback,ModelCheckpoint
from tensorflow.keras.layers import Dense,BatchNormalization,Activation
from tensorflow.keras.activations import relu

sys.path.append('../../')
from Utils.utils import *
from Utils.get_q import *


In [2]:
set = {
  'N': 100,
  'M': 6000,
  'L': 10,
  'ini_type' : 'A',
  'data_name': 'MNIST',


  'data_seed': 4244, #データのサンプリングを固定するためのseed
  'seed1' : 820,
  'seed2' : 314,
}

In [3]:
def PCA_SS(input_train,input_test):
        # Make an instance of the Model
        pca = PCA(n_components=set['N'])
        scaler = StandardScaler()

        train_img = pca.fit_transform(input_train)
        train_img =scaler.fit_transform(train_img)
        
        test_img = pca.transform(input_test)
        test_img =scaler.transform(test_img)
        return train_img, test_img, pca, scaler

In [4]:
set_seed(set['data_seed'])
mnist = tf.keras.datasets.mnist
(X_train, y_train),(X_test, y_test) = mnist.load_data()

In [5]:
print('='*15+'Preprocess data'+'='*15)
X_train, X_test = preprocess_data(X_train, X_test)


idx = np.random.choice(X_train.shape[0], size=set['M'], replace=False)
X_train = X_train[idx]
y_train = y_train[idx]
n_classes = len(np.unique(y_train))

print('N_classes:',n_classes)
print('Train size:',X_train.shape, 'Test size:', y_train.shape)


print('='*15+'PCA'+'='*15)
X_train,X_test, pca1, scaler1 = PCA_SS(X_train, X_test)
print('Train size after PCA:',X_train.shape, 'Test size after PCA:', X_test.shape)

===============Preprocess data===============
N_classes: 10
Train size: (6000, 784) Test size: (6000,)
===============PCA===============
Train size after PCA: (6000, 100) Test size after PCA: (10000, 100)


In [20]:
params= {
  'N': 100,
  'num_classes':10,
  'num_layers':10,
  'activation':'relu',
  'activation_last':'softmax',
  'learning_rate': 0.1,
  'loss': 'sparse_categorical_crossentropy',
  'metric': 'accuracy',
  'random_seed': 777,
  'bias_initializer': tf.keras.initializers.Constant(0.1),
  'optimizer':'adam',
  'epochs':100,
  'batch_size':256
}

In [29]:
def create_fully_model(params:dict, W):
    model=Sequential(name='fully_connect_model')
    model.add(Dense(params['N'], kernel_initializer=W,bias_initializer=params['bias_initializer'],input_shape=(params['N'],), name='Affine1',))
    model.add(BatchNormalization(name='batch_normalization1'))
    model.add(Activation('relu', name='activation1'))
    for i in range(1, params['num_layers']-1):
        model.add(Dense(params['N'], kernel_initializer=W,bias_initializer=params['bias_initializer'],name=f'Affine{i+1}'))
        model.add(BatchNormalization(name=f'batch_normalization{i+1}'))
        model.add(Activation(params['activation'],name=f'activation{i+1}'))
        
    model.add(Dense(params['num_classes'],kernel_initializer=W,bias_initializer=params['bias_initializer'],name='output',))
    model.add(BatchNormalization(name=f'batch_normalization{params["num_layers"]}'))
    model.add(Activation('softmax', name=f'activation{params["num_layers"]}'))
    model.compile(loss=params['loss'], optimizer=params['optimizer'],metrics=[params['metric']])
    return model

In [30]:
w_intializer1 = tf.keras.initializers.RandomNormal(mean=0, stddev=1)
model1 = create_fully_model(params=params,W=w_intializer1)

In [37]:
# モデルのレイヤー名を取得する
layer_names = [layer.name for layer in model1.layers]
print(layer_names)

['Affine1', 'batch_normalization1', 'activation1', 'Affine2', 'batch_normalization2', 'activation2', 'Affine3', 'batch_normalization3', 'activation3', 'Affine4', 'batch_normalization4', 'activation4', 'Affine5', 'batch_normalization5', 'activation5', 'Affine6', 'batch_normalization6', 'activation6', 'Affine7', 'batch_normalization7', 'activation7', 'Affine8', 'batch_normalization8', 'activation8', 'Affine9', 'batch_normalization9', 'activation9', 'output', 'batch_normalization10', 'activation10']


In [28]:
history1 = model1.fit(X_train, y_train,
                batch_size=params["batch_size"],
                epochs=params["epochs"],
                verbose=1,
                shuffle=True,
                validation_data=(X_test, y_test),
                #callbacks=[LogEpochIntermediateCallcack(layer_name_list=CFG.layer_name_list,model_num='001')]
                )

Epoch 1/100


ValueError: in user code:

    File "/Users/oritoyusuke/.pyenv/versions/miniforge3-4.10.1-5/envs/ms-env/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/oritoyusuke/.pyenv/versions/miniforge3-4.10.1-5/envs/ms-env/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/oritoyusuke/.pyenv/versions/miniforge3-4.10.1-5/envs/ms-env/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/oritoyusuke/.pyenv/versions/miniforge3-4.10.1-5/envs/ms-env/lib/python3.8/site-packages/keras/engine/training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/oritoyusuke/.pyenv/versions/miniforge3-4.10.1-5/envs/ms-env/lib/python3.8/site-packages/keras

In [27]:
X_train.shape

(6000, 100)

In [32]:
loss1 = joblib.load('../Output/Loss/model001_iniA_M6000_L10_seed820.pkl')